<a href="https://colab.research.google.com/github/Hwismos/capstone-keras-based-model/blob/main/ncf/NeuMF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 환경 설정

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cd '/content/drive/MyDrive/4학년/캡스톤/[공부] 인공지능 공부/[05.07] Keras-Collaborative Filtering'

# 협업 필터링 신경망 기반의 영화 추천 시스템
- Keras 라이브러리를 이용해 딥러닝 모델 구축

## 소개

In [ ]:
'''
● MovieLens(ML) 데이터셋과 신경망 기반 협업 필터링을 이용해 사용자에게 영화를 추천해주는 모델을 구축한다.
● 이 프로젝트의 목적은 사용자가 레이팅하지 않은 영화에 대한 레이팅 값을 예측하는 것이다.
● 그 결과로, 높게 레이팅된 영화들을 유저에게 추천해준다. 
'''

In [ ]:
!pip install zipfile

In [ ]:
import pandas as pd
import numpy as np
from zipfile import ZipFile
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from pathlib import Path
import matplotlib.pyplot as plt

## 데이터셋 로딩과 전처리

In [ ]:
# 실제 데이터를 다운로드 받는다.
# ratings.csf 파일을 이용한다. 
movielens_data_file_url = (
    "http://files.grouplens.org/datasets/movielens/ml-latest-small.zip"
)

movielens_zipped_file = keras.utils.get_file(
    "ml-latest-small.zip", movielens_data_file_url, extract=False
)

# 반환 타입은 다음과 같다. 
# <class 'pathlib.PosixPath'>
keras_datasets_path = Path(movielens_zipped_file).parents[0]  
movielens_dir = keras_datasets_path / "ml-latest-small"

# 한 번만 실행된다.
if not movielens_dir.exists():
    with ZipFile(movielens_zipped_file, "r") as zip:
        print("Extracting all the files now...")
        zip.extractall(path=keras_datasets_path)
        print("Done!!!")
# 실행되지 않았다.

ratings_file = movielens_dir / "ratings.csv"

# DataFrame을 확인해보면 좋을 것 같다.
df = pd.read_csv(ratings_file)  # DataFrame

In [ ]:
# 데이터프레임 객체를 확인한다.

df.head(10)

In [ ]:
# 유저와 영화에 대한 정수 인덱스들을 인코딩하기 위한 전처리가 필요하다.

user_ids = df["userId"].unique().tolist()  # Series 객체의 unique value들을 list 포맷으로 반환한다.
# 유저 → 인코딩된 유저
user2user_encoded = {x: i for i, x in enumerate(user_ids)}
# 인코딩된 유저 → 유저
user_encoded2user = {i: x for i, x in enumerate(user_ids)}

movie_ids = df["movieId"].unique().tolist()
movie2movie_encoded = {x: i for i, x in enumerate(movie_ids)}
movie_encoded2movie = {i: x for i, x in enumerate(movie_ids)}

# map 메소드의 인자 값에 따라 Series 객체를 매핑한다.
    # 인자는 dictionary 형태이다. 
# 이 과정을 통해 새로운 column을 추가한다.
df["user"] = df["userId"].map(user2user_encoded)
df["movie"] = df["movieId"].map(movie2movie_encoded)

num_users = len(user2user_encoded)
num_movies = len(movie_encoded2movie)
df["rating"] = df["rating"].values.astype(np.float32)

# 레이팅의 최소, 최대값은 정규화에 이용된다.
min_rating = min(df["rating"])
max_rating = max(df["rating"])

print(
    "Number of users: {}, Number of Movies: {}, Min rating: {}, Max rating: {}".format(
        num_users, num_movies, min_rating, max_rating
    )
)

In [ ]:
# check column을 생성해서 모두 1로 설정한다. 
edge = {x: 1.0 for x in user_ids}
df["edge"] = df["userId"].map(edge)
# print(df["edge"])
df.head(10)

In [ ]:
# df로부터 추출한 데이터들의 포맷과 정보를 확인한다.

# print(type(df["userId"]))
# print(type(user_ids))
# print(user_ids)

# user와 movie라는 column이 추가됐다.
df.head(10)

## 학습, 평가 데이터 준비

In [ ]:
df = df.sample(frac=1, random_state=42)  # df를 랜덤하게 섞는다.
x = df[["user", "movie"]].values  # user와 movie 컬럼의 value들을 추출한다.

# 학습의 편의를 위해 레이팅을 0과 1 사이 값으로 정규화한다.
# -------------------------------------------------------------------------------------
y = df["rating"].apply(lambda x: (x - min_rating) / (max_rating - min_rating)).values
# y = df["edge"].to_numpy()  # 1.0으로 구성된 연결 관계를 label 데이터로 이용한다.
# -------------------------------------------------------------------------------------
# 화장품 데이터에 대해 rating이 1로 설정된다면,
# 이때의 rating은 유저가 화장품을 조회했다고 판단할 수 있다.

# 9:1의 비율로 학습과 평가 데이터를 분류한다.
train_indices = int(0.9 * df.shape[0])

x_train, x_val, y_train, y_val = (
    x[:train_indices],
    x[train_indices:],
    y[:train_indices],
    y[train_indices:],    
)

In [ ]:
df.head(10)  # 샘플링된 df를 확인한다.

In [ ]:
print(y)  # 정규화된 rating 값을 출력한다.

In [ ]:
print(df[["user", "movie"]])  # 복수 개의 컬럼을 추출할 때 이차원 배열을 이용한다.

## 모델 생성
- 유저와 영화를 50차원의 벡터로 표현한다.
- 모델은 유저와 영화 임베딩의 내적을 이용해 match score를 계산한다.
- 이때 유저와 영화 각각의 바이어스를 더해서 계산한다.
- match score는 0과 1 사이의 값으로 계산된다. 

In [ ]:
EMBEDDING_SIZE = 50

# keras의 Model 클래스를 상속받는다.
class RecommenderNet(keras.Model):
    def __init__(self, num_users, num_movies, embedding_size, **kwargs):
        super().__init__(**kwargs)
        self.num_users = num_users
        self.num_movies = num_movies
        self.embedding_size = embedding_size
        # 4개의 Embedding 레이어를 생성한다.
        self.user_embedding = layers.Embedding(
            num_users,
            embedding_size,
            embeddings_initializer = "he_normal",
            embeddings_regularizer = keras.regularizers.l2(1e-6),
        )
        self.user_bias = layers.Embedding(num_users, 1)
        self.movie_embedding = layers.Embedding(
            num_movies,
            embedding_size,
            embeddings_initializer = "he_normal",
            embeddings_regularizer = keras.regularizers.l2(1e-6),
        )
        self.movie_bias = layers.Embedding(num_movies, 1)
    
    # call 메소드가 호출되면, 모델을 호출해 새로운 인풋에 대해 텐서 형태의 아웃풋을 반환한다.
    # call 메소드는 직접적으로 호출되지 않는다.
        # Model 클래스를 상속 받았을 때 재정의돼야 하는 메소드이다.
    def call(self, inputs):
        # input의 0번째 column을 인자로 이용해, 
        # 유저 임베딩 레이어로부터 특정 유저의 벡터를 추출한다.
        user_vector = self.user_embedding(inputs[:, 0])
        user_bias = self.user_bias(inputs[:, 0])
        movie_vector = self.movie_embedding(inputs[:, 1])
        movie_bias = self.movie_bias(inputs[:, 1])
        dot_user_movie = tf.tensordot(user_vector, movie_vector, 2)
        # 바이어스를 포함한 모든 요소들을 더한다.
        x = dot_user_movie + user_bias + movie_bias
        # 시그모이드 함수가 레이팅을 0과 1로 스케일링 한다.
        return tf.nn.sigmoid(x)

model = RecommenderNet(num_users, num_movies, EMBEDDING_SIZE)
model.compile(
    loss = tf.keras.losses.BinaryCrossentropy(),
    optimizer = keras.optimizers.Adam(learning_rate = 0.001),
    metrics=['accuracy'],
)

# model.summary  # <bound method Model.summary of <__main__.RecommenderNet object at 0x7f41b0cdf910>>

## 분류한 데이터를 이용한 모델 학습

In [ ]:
# fit 메소드는 input_data(x)와 target_data(y)를 이용해 모델을 훈련시킨다.
# batch_size는 그레디언트가 업데이트될 샘플의 수다.
# validation_data는 에포크마다 모델의 loss를 평가하기 위한 데이터다.
    # 모델은 이 데이터를 훈련에 이용하지 않는다.

history = model.fit(
    x = x_train,
    y = y_train,
    batch_size = 64,
    epochs = 5,
    verbose = 1,
    validation_data = (x_val, y_val),
)

In [ ]:
print(type(history))

In [ ]:
# 훈련과 평가에 대한 손실값을 표시한다.

plt.plot(history.history["loss"])
plt.plot(history.history["val_loss"])
plt.title("model loss")
plt.ylabel("loss")
plt.xlabel("epoch")
plt.legend(["train", "test"], loc="upper left")
plt.show()

In [ ]:
# 정확도를 플롯팅한다.

plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

## 특정 유저에 대한 Top-10 영화 추천

In [ ]:
movie_df = pd.read_csv(movielens_dir / "movies.csv")

# 특정 유저와 유저의 top 추천 영화를 얻는다.
user_id = df.userId.sample(1).iloc[0]
movies_watched_by_user = df[df.userId == user_id]
movies_not_watched = movie_df[
    ~movie_df["movieId"].isin(movies_watched_by_user.movieId.values)
]["movieId"]
movies_not_watched = list(
    set(movies_not_watched).intersection(set(movie2movie_encoded.keys()))
)
movies_not_watched = [[movie2movie_encoded.get(x)] for x in movies_not_watched]
user_encoder = user2user_encoded.get(user_id)
user_movie_array = np.hstack(
    ([[user_encoder]] * len(movies_not_watched), movies_not_watched)
)
ratings = model.predict(user_movie_array).flatten()

print(f"\n{ratings}\n")

top_ratings_indices = ratings.argsort()[-10:][::-1]
recommended_movie_ids = [
    movie_encoded2movie.get(movies_not_watched[x][0]) for x in top_ratings_indices
]

print("Showing recomendations for user: {}".format(user_id))
print("====" * 9)
print("Movies with high ratings from user")
print("----" * 8)
# 유저가 본 영화 중 가장 높은 레이팅 값을 가졌던 5개의 영화를 출력한다. 
top_movies_user = (
    movies_watched_by_user.sort_values(by="rating", ascending=False)
    .head(5)
    .movieId.values
)
movie_df_rows = movie_df[movie_df["movieId"].isin(top_movies_user)]
for row in movie_df_rows.itertuples():
    print(row.title, ": ", row.genres)

print("----" * 8)
print("Top 10 movie recommendations")
print("----" * 8)
# 유저가 보지 않은 영화들에 대해서도 예측을 통해 추천을 할 수 있다. 
recommended_movies = movie_df[movie_df["movieId"].isin(recommended_movie_ids)]
for row in recommended_movies.itertuples():
    print(row.title, ": ", row.genres)

## 모델 저장 및 로드

In [ ]:
tf.keras.saving.save_model(
    model, "./my_model", overwrite=True, save_format="tf"
)

# H5 파일은 사용자 정의 레이어를 저장 파일에 포함하지 않는다.
# H5 포맷은 객체의 설정 값들을 이용해 모델 아키텍처를 저장한다.
# 반면, SavedModel 포맷은 실행 그래프를 저장한다.
    # 따라서 사용자 정의 객체(서브 클래싱)는 SavedModel 포맷으로만 저장할 수 있다.

# tf.keras.saving.save_model(
#     model, "./my_model2", overwrite=True, save_format="h5"
# )

In [ ]:
# SavedModel 포맷으로 저장한 파일을 로드해 모델을 불러온다.
# 폴더를 모두 로드하지 않고 .pb 파일만을 로드할 수는 없다.
    # 파일 시그니처를 찾을 수 없다는 에러가 발생한다. 

loaded_model = tf.keras.models.load_model("./my_model")
reloaded_ratings = loaded_model.predict(user_movie_array).flatten()

print(f"\n{reloaded_ratings}\n")

In [ ]:
# 모델 인풋 텐서를 복기한다.
print(model.inputs)

# Keras GCN
- keras_gcn 라이브러리를 확인한다.

In [ ]:
!pip install keras-gcn

In [ ]:
from tensorflow import keras
from keras_gcn import GraphConv


DATA_DIM = 3

data_layer = keras.layers.Input(shape=(None, DATA_DIM))
edge_layer = keras.layers.Input(shape=(None, None))
conv_layer = GraphConv(
    units=32,
    step_num=1,
)([data_layer, edge_layer])